In [ ]:
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')

We first import the necessary libraries. NLTK, or Natural Language Toolkit, is used for processing the transcripts.


In [ ]:
url = "https://raw.githubusercontent.com/wilfordwoodruff/Main-Data/main/data/derived/derived_data.csv"
df = pd.read_csv(url)
df['Current Location'] = None
df['Mentioning Location'] = None

After reading in our data from a CSV file, we add two new columns to the DataFrame: "Current Location" and "Mentioning Location". These will be filled in later.


In [ ]:
df_places = df[df['Places'].notnull()]
df_places['Locations'] = df_places['Places'].str.split('|')

We create a new DataFrame, df_places, which includes only the rows where 'Places' is not null. We then split the 'Places' column on the '|' character and store the result in a new column, 'Locations'.


In [ ]:
current_prepositions = ['in', 'at', 'on']
mentioning_words = ['from', 'about', 'via', 'send', 'talk']

for index, row in df_places.iterrows():
    sentences = nltk.tokenize.sent_tokenize(row['Text Only Transcript'])
    for sentence in sentences:
        words = nltk.tokenize.word_tokenize(sentence)
        tagged_words = nltk.pos_tag(words)
        word_pos_dict = {word.lower(): pos for word, pos in tagged_words}
        for location in row['Locations']:
            if location in sentence:
                for word in current_prepositions:
                    if word in word_pos_dict and word_pos_dict[word] == 'IN':
                        df_places.at[index, 'Current Location'] = location
                for word in mentioning_words:
                    if word in word_pos_dict and (word_pos_dict[word] == 'IN' or word_pos_dict[word] == 'VB'):
                        df_places.at[index, 'Mentioning Location'] = location

Here, we iterate through each sentence in each transcript. For each sentence, we check if any of our locations are mentioned. If they are, we check whether the location is a current location (based on the use of prepositions) or a mentioned location (based on the use of specific verbs or prepositions). If it's a current location, we add it to the 'Current Location' column, and if it's a mentioned location, we add it to the 'Mentioning Location' column.


In [ ]:
df_places['Current Locations']

In [ ]:
df_places['Mentioning Location']